#### **Install các thư viện cần thiết**

In [ ]:
# !pip install scrapy
# !pip install spider3

#### **Import packages**

In [1]:
import json
import re
import pandas as pd
# import scrapy
# import requests
# from selenium.webdriver.common.by import By
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from lxml import html
import json
import httpx
import urllib.request
from time import sleep

#### **Tạo project với scrapy**

In [ ]:
!scrapy startproject laptopmedia

#### **Viết class để parse url của từng trang**

In [ ]:
import scrapy
import re 

class collect_laptop_url(scrapy.Spider):
  name='laptop_urls' 
  custom_settings = {
        'CONCURRENT_REQUESTS_PER_DOMAIN': 1,
        'DOWNLOAD_DELAY': 1
        }
  handle_httpstatus_list = [403]
  
  def start_requests(self):
    urls = ['https://laptopmedia.com/specs/?q=&hPP=1000&idx=laptops&p=0']
    yield scrapy.Request(url=urls[0], callback = self.parse)
  def parse(self, response):
    for href in response.xpath('//*[@id="results_container"]/div/div/article/a[1]/@href').extract():
        yield {
          'laptop_url': href
        }
    offset = re.findall(r'p=(\d+)', response.url)[0]
    if int(offset) < 20:
      next_url = 'https://laptopmedia.com/specs/?q=&hPP=20&idx=laptops&p=' + str(int(offset)+1) 
      yield response.follow(next_url, callback = self.parse, dont_filter=True)

In [ ]:
!cd laptopmedia/laptopmedia
!scrapy crawl laptop_urls -o dataset/laptop_urls.json

-> Gặp lỗi 403 forbidden và không get được dữ liệu

-> Selenium cũng tương tự, javascript chỉ load sau khi raise exception timeout

#### **Crawl tiki**

In [ ]:
import pandas as pd
def url_generator(page_path, cat_id, page):
    return f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&is_mweb=1&aggregations=2&sort=top_seller&urlKey={page_path}&category={cat_id}&page={page}"

page = httpx.get(url_generator('sach_truyen_tieng_viet', '316', '1')).json()
for i in page['filters']:
    if i['display_name'] == 'Danh Mục Sản Phẩm':
        categories = i['values']
        break

print(categories[1])        
       
df = []
    
all_categories = []
for cat in categories:
    cat_data = httpx.get(url_generator(cat['url_key'], cat['query_value'], '1')).json()
    df.append(pd.DataFrame(cat_data['data']))
    has_child = False
    for i in cat_data['filters']: 
        if i['display_name'] == 'Danh Mục Sản Phẩm': # Nếu chứa danh mục con thì thêm vào
            #print(i['display_name'])
            all_categories.extend(i['values'])
            has_child = True
    
    if not has_child: 
        all_categories.append(cat)
        
print(len(all_categories))


In [ ]:
categories[0]

api limit = 100 -> Mỗi lần request lấy được 100 giá trị</br>
Cần lấy giá trị tối đa của số sách có thể có trong một thể loại 

**Lấy id của các cuốn sách ở từng thể loại**

In [ ]:
# all_categories = pd.DataFrame(all_categories)
# display(all_categories)
id_df = []
def get_books_id(cat):
    n = (cat['count']//100) + 2
    books = {
        'Thể loại': cat['display_value'],
        'Book_id': []
    }
    for i in range(1, n):
        response = httpx.get(url_generator(cat['url_key'], cat['query_value'], i)).json()
        if 'data' in response:
            for book in response['data']:
                books['Book_id'].append({'id': book['id']})
                id_df.append({'id': book['id']})
    
    return books


In [ ]:
books_id_all_cate = []
for cate in all_categories:
    print(cate['display_value'])
    books_id_all_cate.append(get_books_id(cate))

In [ ]:
# books_id = pd.DataFrame(books_id_all_cate)
# books_id.to_csv('books_id.csv', encoding='utf-8-sig')
id_df = pd.DataFrame(id_df)
with open('id_data/id_df.json', 'w', encoding='utf-8-sig') as file:
    id_df.to_json('id_data/id_df.json', force_ascii=False, orient='records')
    

### **Lấy thông tin của từng cuốn sách dựa vào id đã thu thập**

In [2]:
try:
    with open('id_data/id_df.json') as f:
        id = json.load(f)
except IOError:
    print("File not found")
    
books_id = [i.get('id') for i in id]


In [3]:

def get_book_info(raw_book):
    book_info = {
                'id': raw_book.get('id'), 
                'master_id': raw_book.get('master_id'),
                'sku': raw_book.get('sku'),
                'name': raw_book.get('name'),
                'short_url': raw_book.get('short_url'),
                #'short_description': raw_book.get('short_description'),
                'book_cover': raw_book.get('book_cover'),
                'price': raw_book.get('price'),
                'original_price': raw_book.get('original_price'),
                'discount_rate': raw_book.get('discount_rate'),
                'rating_average': raw_book.get('rating_average'),
                'review_count': raw_book.get('review_count'),
                'inventory_type': raw_book.get('inventory_type'),
                'productset_group_name': raw_book.get('productset_group_name'),
                'day_ago_created': raw_book.get('day_ago_created'),
                'categories': raw_book.get('categories').get('name') if raw_book.get('categories') not in [None,[]] else None,
                'all_time_quantity_sold': raw_book.get('all_time_quantity_sold', 0)
                }
    
    # authors_name = []
    # authors_name = []
    if raw_book.get('authors') not in [None, []] :
        authors_name = [i.get('name') for i in raw_book.get('authors')]
        book_info['authors'] = ', '.join(authors_name)
    else: book_info['authors'] = ''

    if raw_book.get('specifications') not in [None, []]:
        # print(raw_book['specifications']==[])
        for i in raw_book['specifications'][0]['attributes']:
            # print(i)
            book_info[i['code']] = i['value']
    return book_info

def split_books_id(books_id: list, i, n): # Part 0: i = 0
    length_path = len(books_id)//n
    start = length_path*i
    end = length_path+length_path*i
    if i == n - 1:
        end = len(books_id)
    return start, end

def get_all_books(book_df, start, end):
    for id in books_id[start:end]:
        url = f'https://tiki.vn/api/v2/products/{id}'
        try:
            response = httpx.get(url)
            raw_book = response.json()
            print(response.status_code)
            if response.status_code == 200:
                book_info = get_book_info(raw_book)
                book_df.append(book_info)              
        except:
            book_df.append(get_book_info(raw_book))
            print(f'ERROR:', id)
    return book_df
            


**Vì lượng dữ liệu khá lớn và khó xử lý nên ta sẽ chia thành các phần nhỏ sau đó concat lại:**

In [ ]:
# start, end = split_books_id(books_id, i=0, n=5)
# book_df1 = get_all_books(book_df, start, end)
# df1 = pd.DataFrame(book_df1)
# df1.to_csv('full_data/part_1')

Quên encoding :(

In [ ]:
df = pd.read_csv('full_data/part_1.csv', encoding='utf-8')
df.to_csv('full_data/part_1.csv', encoding='utf-8-sig', index=False)

Tiếp tục với part 2, 3, ..

In [4]:
all_df = []
for i in range(2, 5):
    book_df = []
    start, end = split_books_id(books_id, i, n=5)
    book_df = get_all_books(book_df, start, end)
    df = pd.DataFrame(book_df)
    df.to_csv(f'full_data/part_{i+1}.csv', encoding='utf-8-sig')
    all_df.append(df)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
ERROR: 8458234
ERROR: 198355492
ERROR: 9980608
ERROR: 197738774
ERROR: 166643015
ERROR: 444032
ERROR: 54502150
ERROR: 11728304
ERROR: 34750940
ERROR: 12289258
ERROR: 376156
ERROR: 195624192
ERROR: 7982489
ERROR: 203957144
ERROR: 365879
ERROR: 51232678
ERROR: 174122645
ERROR: 200520507
ERROR: 105127115
ERROR: 202041866
ERROR: 47184498
ERROR: 897744
ERROR: 54431687
ERROR: 365878
ERROR: 155907711
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200

In [4]:
book_df = []
start, end = split_books_id(books_id, 4, n=5)
book_df = get_all_books(book_df, start, end)
df = pd.DataFrame(book_df)
df.to_csv(f'full_data/part_5.csv', encoding='utf-8-sig')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
ERROR: 113527215
ERROR: 164564981
200
ERROR: 42470296
200
200
200
ERROR: 120471847
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
ERROR: 84565190
200
200
200
200
200
200
200
200
200
ERROR: 202961762
200
200
200
200
200
200
ERROR: 189476103
200
ERROR: 202780390
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
20

Mỗi part mất khoảng 150 phút 😅

***

### **Crawl 20 thể loại đầu tiên**

In [ ]:
book_df = []

for cate in books_id_all_cate[:20]:
    for book in cate['Book_id']:
        url = f'https://tiki.vn/api/v2/products/{book["id"]}'
        try:
            response = httpx.get(url)
            raw_book = response.json()
            print(response.status_code)
            if response.status_code == 200:
                book_info = {
                    'id': raw_book['id'],
                    'master_id': raw_book['master_id'],
                    'sku': raw_book['sku'],
                    'name': raw_book['name'],
                    'short_url': raw_book['short_url'],
                    'short_description': raw_book['short_description'],
                    'book_cover': raw_book['book_cover'],
                    'price': raw_book['price'],
                    'original_price': raw_book['original_price'],
                    'discount_rate': raw_book['discount_rate'],
                    'rating_average': raw_book['rating_average'],
                    'review_count': raw_book['review_count'],
                    'inventory_type': raw_book['inventory_type'],
                    'productset_group_name': raw_book['productset_group_name'],
                    'day_ago_created': raw_book['day_ago_created'],
                    'categories': raw_book['categories']['name'],
                    'all_time_quantity_sold': raw_book['all_time_quantity_sold'] if 'all_time_quantity_sold' in raw_book else 0
                    
                }
                name_authors = []
                if 'authors' in raw_book:
                    for author in raw_book['authors']:
                        name_authors.append(author['name'])
                book_info['authors'] = ', '.join(name_authors) 
                
                if 'specifications' in raw_book:
                    for i in raw_book['specifications'][0]['attributes']:
                        book_info[i['code']] = i['code']
                        book_info[i['value']] = i['value']
                book_df.append(book_info)              
        except:
            book_df.append(raw_book)
            print('ERROR:', book['id'])

<font color=red>Mất khoảng 185 phút cho 1/5 dữ liệu 😅</font>

In [ ]:
count = 0
for i in books_id_all_cate[:20]:
    count+= len(i['Book_id'])
print(count)

**Tổng số cuốn sách phải crawl**

In [ ]:
count = 0
id = []
for i in books_id_all_cate:
    count+= len(i['Book_id'])
    for j in i['Book_id']:
        id.append(j['id'])
print(count)
print(len(set(id)))

**20 thể loại đầu tiên**

In [ ]:
book_df = pd.DataFrame(book_df)
book_df.to_csv('books_data1.csv', encoding='utf-8-sig')
print(book_df.shape)
display(book_df.tail())

In [ ]:
book_df.duplicated().sum()

In [ ]:
book_df.drop_duplicates()

### **Thử test tốc độ crawl**

In [ ]:
book_test = []

def get_book_info(raw_book, book):
    book_info = {
                'id': raw_book['id'], 
                'master_id': raw_book['master_id'],
                'sku': raw_book['sku'],
                'name': raw_book['name'],
                'short_url': raw_book['short_url'],
                'short_description': raw_book['short_description'],
                'book_cover': raw_book['book_cover'],
                'price': raw_book['price'],
                'original_price': raw_book['original_price'],
                'discount_rate': raw_book['discount_rate'],
                'rating_average': raw_book['rating_average'],
                'review_count': raw_book['review_count'],
                'inventory_type': raw_book['inventory_type'],
                'productset_group_name': raw_book['productset_group_name'],
                'day_ago_created': raw_book['day_ago_created'],
                'categories': raw_book['categories']['name'] if 'categories' in book else [],
                'all_time_quantity_sold': raw_book['all_time_quantity_sold'] if 'all_time_quantity_sold' in book else 0

                }
    name_authors = []
    if 'authors' in raw_book:
        for author in raw_book['authors']:
            name_authors.append(author['name'])
    book_info['authors'] = ', '.join(name_authors)

    if 'specifications' in book:
        for i in book['specifications'][0]['attributes']:
            book_info[i['code']] = i['code']
            book_info[i['value']] = i['value']
    return book_info

for cate in books_id_all_cate[:1]:
    for book in cate['Book_id']:
        url = f'https://tiki.vn/api/v2/products/{book["id"]}'
        try:
            response = httpx.get(url)
            print(response.status_code)
            if response.status_code == 200:
                raw_book = response.json()
                book_test.append(get_book_info(raw_book, book))              
        except:
            # book_test.append(raw_book)
            print(f'ERROR {response.status_code}:', book)
    

**Có vẻ như ta đã lấy được hết các dòng**

In [ ]:
len(book_test)

In [ ]:
count = 0
book_test_copy = book_test.copy()
for i in book_test:
    if i!= None:
        count+=1
    else: book_test_copy.remove(i)

len(book_test_copy)

**Có khoảng 195 id và ta lấy được hết**

In [ ]:
assert len(book_test) == len(books_id_all_cate[0]['Book_id'])

**Thử lại với multithread**

In [ ]:
import concurrent.futures 
import time

NUMBER_OF_WORKERS = 4
        
def handle_get_book_info(book):
    url = f'https://tiki.vn/api/v2/products/{book["id"]}'
    try:    
        response = httpx.get(url)   
        print(response.status_code)
        if response.status_code == 200: #and 'content-type' in response.headers 
            #and 'application/json' in response.headers['content-type']):
            raw_book = response.json()
            
            book_info = get_book_info(raw_book, book)
            return book_info
        # response.raise_for_status()
        
    except Exception as e:
        # df.append(book)
        # book_info = response.json()
        # if response.status_code == 429:
        #     time.sleep(int(response.headers["Retry-After"]))
        print(f'ERROR {str(e)}: ', book['id'])
        # elif response.status_code == 200:
        #     book_info = get_book_info(response.json(), book)
        # book_info = raw_book
        # return book_info
        # return response.json()
    # return book_info
        
test_df = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUMBER_OF_WORKERS) as executor:
    all_params = list(book for cate in books_id_all_cate[:1] 
                      for book in cate['Book_id'] )
    for future in concurrent.futures.as_completed(
        executor.submit(handle_get_book_info, params)
        for params in all_params
    ):
        # result = future.result()
        test_df.append(future.result())
        #print(result)

**Với 10 threads ta lấy được khoảng 130 dòng, tốc độ tăng lên đáng kể: 1m25s -> 25s**
<br>**Tiếp tục thử lại với 5 threads**

|Threads|Số dòng lấy được|Tốc độ|
| -- | -- | -- |
|10|130|25s|
|5|140|38s|
|2|195|59s|

429 error -> too many requests

In [ ]:
# len(test_df)
count = 0
test = test_df.copy()
for i in test_df:
    if i!= None:
        # print(i['id'])
        count+=1
    else: 
        test.remove(i)
len(test)

**Vậy ta sẽ thử trên toàn bộ dữ liệu với 2 threads**

In [ ]:
test_df_second = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUMBER_OF_WORKERS) as executor:
    all_params = list(book for cate in books_id_all_cate[:20] 
                      for book in cate['Book_id'] )
    for future in concurrent.futures.as_completed(
        executor.submit(handle_get_book_info, params)
        for params in all_params
    ):
        test_df_second.append(future.result())